# LSPP Assignment - Intelligent Agent: Information Extraction

This notebook extracts structured company information from essays using LangChain's LCEL (LangChain Expression Language) and Google's Gemini API. It processes text paragraph-by-paragraph to extract:

- **Company Name**: The name of the company
- **Founding Date**: When the company was founded (normalized to YYYY-MM-DD format)
- **Founders**: List of founders (real human names only)

## Features

- 📝 Paragraph-wise extraction using LCEL
- 📅 Smart date normalization (handles various formats like "2010", "March 2010", "March 15, 2010")
- 📊 CSV output with proper formatting
- 🔄 Rate limiting to avoid API quota issues
- 🔍 Preview functionality to see results before saving

## Setup Requirements

- Google Colab environment
- Gemini API key (stored in Colab secrets)
- Internet connection for package installation

# 🎯 How to Use This Notebook

### Step 1: Setup
1. **Add your Gemini API key** to Google Colab secrets:
   - Click the 🔑 icon in the left sidebar
   - Add a new secret named `GEMINI_API_KEY`
   - Paste your API key as the value

### Step 2: Run the Cells
1. **Run all cells** from top to bottom (Runtime → Run all)
2. The notebook will automatically process the sample essay text
3. Results will be displayed and saved to `company_info.csv`

### Step 3: Customize (Optional)

#### Use Your Own Text
Replace the `SAMPLE_ESSAY` variable with your own text:

```python
# Replace SAMPLE_ESSAY with your text
SAMPLE_ESSAY = """Your essay text here..."""
```

#### Upload a Text File
```python
# Upload and read a text file
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]
with open(filename, 'r', encoding='utf-8') as f:
    SAMPLE_ESSAY = f.read()
```

#### Adjust Configuration
Modify the `CONFIG` dictionary to customize behavior:

```python
CONFIG = {
    'delay_sec': 2.0,          # Increase for stricter rate limiting
    'max_paragraphs': 5,       # Limit for testing (None = all)
    'show_preview': True,      # Show results preview
    'output_csv': 'my_companies.csv'  # Custom output filename
}
```

### Output Format

The CSV file contains these columns:
- **S.N.**: Serial number
- **Company Name**: Extracted company name
- **Founded in**: Founding date (YYYY-MM-DD format)
- **Founded by**: List of founders (JSON format)

### Troubleshooting

- **API Key Error**: Make sure your `GEMINI_API_KEY` is correctly set in Colab secrets
- **Rate Limit Error**: Increase `delay_sec` in the configuration
- **No Results**: Check if your text contains clear company founding information
- **Date Format Issues**: The system handles various date formats automatically

### Download Results

```python
# Download the CSV file
from google.colab import files
files.download('company_info.csv')
```

## 1. Install Required Dependencies

First, we'll install all the necessary packages for our company information extraction project.

In [ ]:
# Install required packages
!pip install langchain-google-genai pydantic python-dotenv python-dateutil langchain-core langchain -qU

## 2. Import Libraries and Setup

Import all required libraries and configure Google Colab secrets to access the Gemini API key.

In [ ]:
from __future__ import annotations
import os
import re
import csv
import json
import time
from datetime import date
from typing import List, Union
from pathlib import Path

# Pydantic imports
from pydantic import BaseModel, Field, field_validator

# LangChain imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

# Date parsing
from dateutil import parser as date_parser

# Google Colab secrets
from google.colab import userdata

# Setup API key from Google Colab secrets
# Make sure to add your GEMINI_API_KEY in Colab secrets (🔑 icon in the sidebar)
try:
    api_key = userdata.get('GEMINI_API_KEY')
    os.environ['GOOGLE_API_KEY'] = api_key
    print("✅ API key loaded successfully from Colab secrets")
except Exception as e:
    print(f"❌ Error loading API key: {e}")
    print("Please add your GEMINI_API_KEY to Colab secrets (🔑 icon in sidebar)")

## 3. Define Pydantic Schemas with Date Normalization

Create data models for company information with smart date parsing that handles various date formats.

In [ ]:
# Month mapping for date parsing
MONTHS = {m.lower(): i for i, m in enumerate([
    "", "January","February","March","April","May","June","July","August","September","October","November","December"
])}

def normalize_partial_date(text: str) -> date:
    """
    Normalize various date formats to a standard date object.
    Handles formats like:
    - YYYY (e.g., "2010") → 2010-01-01
    - Month YYYY (e.g., "March 2010") → 2010-03-01
    - YYYY-MM-DD (e.g., "2010-03-15") → 2010-03-15
    - Month D, YYYY (e.g., "March 15, 2010") → 2010-03-15
    """
    s = text.strip()

    # Handle full date format YYYY-MM-DD
    m_full = re.fullmatch(r"(\d{4})-(\d{2})-(\d{2})", s)
    if m_full:
        y, mo, d = map(int, m_full.groups())
        return date(y, mo, d)

    # Try parsing with dateutil for complex formats
    try:
        dt = date_parser.parse(s, default=date(1900, 1, 1), dayfirst=False, fuzzy=True)

        # Handle year-only format (e.g., "2010")
        year_match = re.fullmatch(r"\d{4}", s)
        if year_match:
            return date(int(year_match.group(0)), 1, 1)

        # Handle Month YYYY format (e.g., "March 2010")
        month_year = re.fullmatch(r"([A-Za-z]+)\s+(\d{4})", s)
        if month_year:
            mo = MONTHS.get(month_year.group(1).lower(), 1)
            return date(int(month_year.group(2)), mo, 1)

        # Use parsed date
        return date(dt.year, dt.month, dt.day)
    except Exception:
        # Fallback: extract year and try to find month
        m_year = re.search(r"(\d{4})", s)
        if m_year:
            y = int(m_year.group(1))
            # Try to find month name
            m_name = re.search(r"(January|February|March|April|May|June|July|August|September|October|November|December)", s, re.IGNORECASE)
            if m_name:
                mo = MONTHS[m_name.group(1).lower()]
                return date(y, mo, 1)
            return date(y, 1, 1)
        raise ValueError(f"Unrecognized date format: {text}")

class CompanyInfo(BaseModel):
    """Model for individual company information."""
    company_name: str
    founding_date: date
    founders: List[str]

    @field_validator("founding_date", mode="before")
    @classmethod
    def _parse_date(cls, v: Union[str, date]):
        if isinstance(v, date):
            return v
        return normalize_partial_date(str(v))

class ParagraphExtraction(BaseModel):
    """Model for extraction results from a single paragraph."""
    items: List[CompanyInfo]

# Test the date normalization function
test_dates = ["2010", "March 2010", "2010-03-15", "March 15, 2010", "April 4, 1975"]
print("📅 Date Normalization Examples:")
for test_date in test_dates:
    try:
        normalized = normalize_partial_date(test_date)
        print(f"  '{test_date}' → {normalized.isoformat()}")
    except Exception as e:
        print(f"  '{test_date}' → Error: {e}")

## 4. Build LLM and Text Processing Functions

Set up the Gemini LLM instance and create functions to process text.

In [ ]:
def build_llm(model_name: str = None, temperature: float = 0.0):
    """Build and configure the Gemini LLM instance."""
    api_key = os.getenv("GOOGLE_API_KEY")
    if not api_key:
        raise RuntimeError("GOOGLE_API_KEY is not set. Please check your Colab secrets.")

    api_key = api_key.strip()
    model = model_name or "gemini-1.5-flash"  # Use flash model for lower quota usage

    return ChatGoogleGenerativeAI(
        model=model,
        temperature=temperature,
        api_key=api_key
    )

def split_into_paragraphs(text: str) -> List[str]:
    """Split text into individual paragraphs for processing."""
    paras = [p.strip() for p in re.split(r"\n\s*\n", text) if p.strip()]
    return paras

# Test the functions
print("🤖 Testing LLM connection...")
try:
    llm = build_llm()
    print("✅ LLM connection successful!")
except Exception as e:
    print(f"❌ LLM connection failed: {e}")

# Test paragraph splitting
sample_text = """This is paragraph one.
It has multiple lines.

This is paragraph two.
With different content.

And this is paragraph three."""

paragraphs = split_into_paragraphs(sample_text)
print(f"\n📝 Paragraph splitting test:")
print(f"  Input: {len(sample_text)} characters")
print(f"  Output: {len(paragraphs)} paragraphs")
for i, para in enumerate(paragraphs, 1):
    print(f"    {i}. {para[:50]}{'...' if len(para) > 50 else ''}")

## 5. Create Extraction Chain and Processing Logic

Build the LCEL chain for extracting company information and implement the main processing logic.

In [ ]:
def _build_one_paragraph_chain():
    """Build the LCEL chain for processing a single paragraph."""
    llm = build_llm()

    # Create the prompt template
    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "Extract company founding facts from the paragraph. "
         "Return items as a JSON object matching the schema 'ParagraphExtraction'. "
         "Each item must include: company_name (string), founding_date (string, can be YYYY, YYYY-MM or YYYY-MM-DD), "
         "and founders (list of strings; only real human names). "
         "If multiple companies are present, include them all. "
         "If no companies are found, return an empty list."),
        ("human", "{paragraph}")
    ])

    # Create structured output chain
    structured = llm.with_structured_output(ParagraphExtraction)

    # Return the complete chain
    return prompt | structured

def run_extraction(text: str, delay_sec: float = 1.0, max_paragraphs: int = None) -> List[CompanyInfo]:
    """
    Sequentially process each paragraph to avoid rate limits and return extracted companies.

    Args:
        text: Input text to process
        delay_sec: Delay between API calls to avoid rate limits
        max_paragraphs: Maximum number of paragraphs to process (for testing)

    Returns:
        List of extracted CompanyInfo objects
    """
    print("🔄 Starting extraction process...")

    # Split text into paragraphs
    paragraphs = split_into_paragraphs(text)
    if max_paragraphs:
        paragraphs = paragraphs[:max_paragraphs]
        print(f"📝 Limited to first {max_paragraphs} paragraphs")

    print(f"📄 Processing {len(paragraphs)} paragraphs...")

    # Build the extraction chain
    extraction_chain = _build_one_paragraph_chain()

    # Process each paragraph
    all_items: List[CompanyInfo] = []
    results: List[ParagraphExtraction] = []

    for i, paragraph in enumerate(paragraphs, 1):
        print(f"  Processing paragraph {i}/{len(paragraphs)}...")
        try:
            # Extract companies from this paragraph
            result: ParagraphExtraction = extraction_chain.invoke({"paragraph": paragraph})
            results.append(result)

            # Add extracted companies to our list
            if result.items:
                all_items.extend(result.items)
                print(f"    ✅ Found {len(result.items)} companies")
            else:
                print(f"    ℹ️ No companies found")

        except Exception as e:
            print(f"    ❌ Error processing paragraph {i}: {e}")

        # Rate limiting delay
        if delay_sec and i < len(paragraphs):
            print(f"    ⏳ Waiting {delay_sec}s to avoid rate limits...")
            time.sleep(delay_sec)

    print(f"✅ Extraction complete! Found {len(all_items)} total companies.")
    return all_items

# Test with a small sample
test_text = """
Microsoft Corporation was founded in 1975 by Bill Gates and Paul Allen. The company started as a small software company.

Apple Inc. was founded on April 1, 1976 by Steve Jobs, Steve Wozniak, and Ronald Wayne. The company revolutionized personal computing.

Google was founded in September 1998 by Larry Page and Sergey Brin while they were PhD students at Stanford University.
"""

print("🧪 Testing extraction with sample text...")
try:
    test_results = run_extraction(test_text, delay_sec=0.5, max_paragraphs=2)
    print(f"\n📊 Test Results:")
    for i, company in enumerate(test_results, 1):
        print(f"  {i}. {company.company_name}")
        print(f"     Founded: {company.founding_date.isoformat()}")
        print(f"     Founders: {', '.join(company.founders)}")
except Exception as e:
    print(f"❌ Test failed: {e}")

## 6. Define CSV Writing Functions

Create functions to write extracted company data to CSV format with proper formatting.

In [ ]:
def append_csv(csv_path: str, company_name: str, founding_date: str, founders: List[str]) -> str:
    """
    Append a company entry to CSV file with proper serial numbering.

    Args:
        csv_path: Path to the CSV file
        company_name: Name of the company
        founding_date: Founding date in YYYY-MM-DD format
        founders: List of founder names

    Returns:
        Status message
    """
    path = Path(csv_path)
    new_file = not path.exists()

    # Create directory if it doesn't exist
    path.parent.mkdir(parents=True, exist_ok=True)

    # Calculate serial number
    sn = 1
    if not new_file:
        try:
            with path.open("r", encoding="utf-8") as r:
                rows = list(csv.reader(r))
            sn = len(rows)  # header is row 1, so this gives us the next S.N.
        except Exception:
            sn = 1

    # Write to CSV
    with path.open("a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)

        # Write header for new file
        if new_file:
            writer.writerow(["S.N.", "Company Name", "Founded in", "Founded by"])

        # Convert founders list to JSON string for CSV storage
        founders_str = json.dumps(founders, ensure_ascii=False)

        # Write the data row
        writer.writerow([sn, company_name, founding_date, founders_str])

    return f"Appended {company_name} to {csv_path}"

def write_csv(items: List[CompanyInfo], out_csv: str) -> str:
    """
    Write all extracted companies to a CSV file.

    Args:
        items: List of CompanyInfo objects
        out_csv: Output CSV file path

    Returns:
        Status message
    """
    path = Path(out_csv)

    # Remove existing file to start fresh
    if path.exists():
        path.unlink()

    # Write each company using append_csv to ensure proper formatting
    for item in items:
        append_csv(
            csv_path=out_csv,
            company_name=item.company_name,
            founding_date=item.founding_date.isoformat(),
            founders=item.founders,
        )

    return f"Successfully wrote {len(items)} companies to {out_csv}"

def preview_results(items: List[CompanyInfo], max_items: int = 10):
    """Display a preview of extracted companies."""
    print(f"📊 Preview of extracted companies (showing {min(len(items), max_items)} of {len(items)}):")
    print("-" * 80)

    for i, item in enumerate(items[:max_items], 1):
        print(f"{i:2d}. Company: {item.company_name}")
        print(f"     Founded: {item.founding_date.isoformat()}")
        print(f"     Founders: {', '.join(item.founders)}")
        print()

    if len(items) > max_items:
        print(f"... and {len(items) - max_items} more companies")

# Test CSV functions with sample data
print("🧪 Testing CSV functions...")
sample_companies = [
    CompanyInfo(company_name="Test Corp", founding_date="2020-01-01", founders=["John Doe", "Jane Smith"]),
    CompanyInfo(company_name="Example Inc", founding_date="2021-06-15", founders=["Bob Johnson"])
]

test_csv_path = "test_companies.csv"
try:
    result = write_csv(sample_companies, test_csv_path)
    print(f"✅ {result}")

    # Read and display the CSV content
    with open(test_csv_path, 'r', encoding='utf-8') as f:
        content = f.read()
    print(f"\n📄 CSV Content:")
    print(content)

except Exception as e:
    print(f"❌ CSV test failed: {e}")

## 7. Main Execution and Results Display

Now let's put everything together and run the complete extraction pipeline.

In [ ]:
# Sample essay text for demonstration

SAMPLE_ESSAY = """
In the ever-evolving landscape of global commerce, the origin stories of major corporations are not merely tales of personal ambition and entrepreneurial spirit but also reflections of broader socio-economic trends and technological revolutions that have reshaped industries. These narratives, which often begin with modest ambitions, unfold into chronicles of innovation and strategic foresight that define industries and set benchmarks for future enterprises.
Early Foundations: Pioneers of Industry
One of the earliest examples is The Coca-Cola Company, founded on May 8, 1886, by Dr. John Stith Pemberton in Atlanta, Georgia. Initially sold at Jacob's Pharmacy as a medicinal beverage, Coca-Cola would become one of the most recognized brands worldwide, revolutionizing the beverage industry.
Similarly, Sony Corporation was established on May 7, 1946, by Masaru Ibuka and Akio Morita in Tokyo, Japan. Starting with repairing and building electrical equipment in post-war Japan, Sony would grow to pioneer electronics, entertainment, and technology.
As the mid-20th century progressed, McDonald's Corporation emerged as a game-changer in the fast-food industry. Founded on April 15, 1955, in Des Plaines, Illinois, by Ray Kroc, McDonald's built upon the original concept of Richard and Maurice McDonald to standardize and scale fast-food service globally. Around the same period, Intel Corporation was established on July 18, 1968, by Robert Noyce and Gordon Moore in Mountain View, California driving advancements in semiconductors and microprocessors that became the backbone of modern computing.
The Rise of Technology Titans
Samsung Electronics Co., Ltd., founded on January 13, 1969, by Lee Byung-chul in Su-dong, South Korea, initially focused on producing electrical appliances like televisions and refrigerators. As Samsung expanded into semiconductors, telecommunications, and digital media, it grew into a global technology leader. Similarly, Microsoft Corporation was founded on April 4, 1975, by Bill Gates and Paul Allen in Albuquerque, New Mexico, with the vision of placing a computer on every desk and in every home.
In Cupertino, California, Apple Inc. was born on April 1, 1976, founded by Steve Jobs, Steve Wozniak, and Ronald Wayne. Their mission to make personal computing accessible and elegant revolutionized technology and design. A few years later, Oracle Corporation was established on June 16, 1977, by Larry Ellison, Bob Miner, and Ed Oates in Santa Clara, California. Specializing in relational databases, Oracle would become a cornerstone of enterprise software and cloud computing.
NVIDIA Corporation, founded on April 5, 1993, by Jensen Huang, Chris Malachowsky, and Curtis Priem in Santa Clara, California, began with a focus on graphics processing units (GPUs) for gaming. Today, NVIDIA is a leader in artificial intelligence, deep learning, and autonomous systems, showcasing the power of continuous innovation.
E-Commerce and the Internet Revolution
The 1990s witnessed a dramatic shift toward e-commerce and internet technologies. Amazon.com Inc. was founded on July 5, 1994, by Jeff Bezos in a garage in Bellevue, Washington, with the vision of becoming the world's largest online bookstore. This vision rapidly expanded to encompass e-commerce, cloud computing, and digital streaming. Similarly, Google LLC was founded on September 4, 1998, by Larry Page and Sergey Brin, PhD students at Stanford University, in a garage in Menlo Park, California. Google's mission to "organize the world's information" transformed how we search, learn, and connect.
In Asia, Alibaba Group Holding Limited was founded on June 28, 1999, by Jack Ma and 18 colleagues in Hangzhou, China. Originally an e-commerce platform connecting manufacturers with buyers, Alibaba expanded into cloud computing, digital entertainment, and financial technology, becoming a global powerhouse.
In Europe, SAP SE was founded on April 1, 1972, by Dietmar Hopp, Hans-Werner Hector, Hasso Plattner, Klaus Tschira, and Claus Wellenreuther in Weinheim, Germany. Specializing in enterprise resource planning (ERP) software, SAP revolutionized how businesses manage operations and data.
Social Media and Digital Platforms
The 2000s brought a wave of social media and digital platforms that reshaped communication and commerce. LinkedIn Corporation was founded on December 28, 2002, by Reid Hoffman and a team from PayPal and Socialnet.com in Mountain View, California, focusing on professional networking. Facebook, Inc. (now Meta Platforms, Inc.) was launched on February 4, 2004, by Mark Zuckerberg and his college roommates in Cambridge, Massachusetts, evolving into a global social networking behemoth.
Another transformative platform, Twitter, Inc., was founded on March 21, 2006, by Jack Dorsey, Biz Stone, and Evan Williams in San Francisco, California. Starting as a microblogging service, Twitter became a critical tool for communication and social commentary. Spotify AB, founded on April 23, 2006, by Daniel Ek and Martin Lorentzon in Stockholm, Sweden, leveraged streaming technology to democratize music consumption, fundamentally altering the music industry.
In the realm of video-sharing, YouTube LLC was founded on February 14, 2005, by Steve Chen, Chad Hurley, and Jawed Karim in San Mateo, California. YouTube became the leading platform for user-generated video content, influencing global culture and media consumption.
Innovators in Modern Technology
Tesla, Inc., founded on July 1, 2003, by a group including Elon Musk, Martin Eberhard, Marc Tarpenning, JB Straubel, and Ian Wright, in San Carlos, California, championed the transition to sustainable energy with its electric vehicles and energy solutions. Airbnb, Inc., founded in August 2008 by Brian Chesky, Joe Gebbia, and Nathan Blecharczyk in San Francisco, California, disrupted traditional hospitality with its peer-to-peer lodging platform.
In the realm of fintech, PayPal Holdings, Inc. was established in December 1998 by Peter Thiel, Max Levchin, Luke Nosek, and Ken Howery in Palo Alto, California. Originally a cryptography company, PayPal became a global leader in online payments. Stripe, Inc., founded in 2010 by Patrick and John Collison in Palo Alto, California, followed suit, simplifying online payments and enabling digital commerce.
Square, Inc. (now Block, Inc.), founded on February 20, 2009, by Jack Dorsey and Jim McKelvey in San Francisco, California, revolutionized mobile payment systems with its simple and accessible card readers.
Recent Disruptors
Zoom Video Communications, Inc. was founded on April 21, 2011, by Eric Yuan in San Jose, California. Initially designed for video conferencing, Zoom became essential during the COVID-19 pandemic, transforming remote work and communication. Slack Technologies, LLC, founded in 2009 by Stewart Butterfield, Eric Costello, Cal Henderson, and Serguei Mourachov in Vancouver, Canada, redefined workplace communication with its innovative messaging platform.
Rivian Automotive, Inc., founded on June 23, 2009, by RJ Scaringe in Plymouth, Michigan, entered the electric vehicle market with a focus on adventure and sustainability. SpaceX, established on March 14, 2002, by Elon Musk in Hawthorne, California, revolutionized aerospace with reusable rockets and ambitious plans for Mars exploration.
TikTok, developed by ByteDance and launched in September 2016 by Zhang Yiming in Beijing, China, revolutionized short-form video content, becoming a cultural phenomenon worldwide.
Conclusion
These corporations, with their diverse beginnings and visionary founders, exemplify the interplay of innovation, timing, and strategic foresight that shapes industries and transforms markets. From repairing electronics in post-war Japan to building global e-commerce empires and redefining space exploration, their stories are milestones in the narrative of global economic transformation. Each reflects not only the aspirations of their founders but also the technological advancements and socio-economic trends of their time, serving as inspirations for future innovators.
"""

print("📄 Sample Essay Text Loaded")
print(f"📊 Text Statistics:")
print(f"  Length: {len(SAMPLE_ESSAY):,} characters")
print(f"  Paragraphs: {len(split_into_paragraphs(SAMPLE_ESSAY))} paragraphs")
print(f"  Words: {len(SAMPLE_ESSAY.split()):,} words")

In [ ]:
# Configuration settings
CONFIG = {
    'delay_sec': 2.0,          # Delay between API calls (increase if hitting rate limits)
    'max_paragraphs': None,    # Limit paragraphs for testing (None = process all)
    'show_preview': True,      # Show preview of results
    'output_csv': 'company_info.csv'  # Output CSV filename
}

print("⚙️ Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

print("\n" + "="*60)
print("🚀 STARTING COMPANY EXTRACTION PROCESS")
print("="*60)

In [ ]:
# Run the extraction process
try:
    # Extract companies from the essay
    extracted_companies = run_extraction(
        text=SAMPLE_ESSAY,
        delay_sec=CONFIG['delay_sec'],
        max_paragraphs=CONFIG['max_paragraphs']
    )

    print(f"\n✅ Extraction completed successfully!")
    print(f"📊 Total companies extracted: {len(extracted_companies)}")

    # Show preview if enabled
    if CONFIG['show_preview'] and extracted_companies:
        print(f"\n" + "="*60)
        preview_results(extracted_companies)
        print("="*60)

    # Write to CSV
    if extracted_companies:
        csv_result = write_csv(extracted_companies, CONFIG['output_csv'])
        print(f"\n💾 {csv_result}")

        # Display CSV content
        try:
            with open(CONFIG['output_csv'], 'r', encoding='utf-8') as f:
                csv_content = f.read()
            print(f"\n📄 Generated CSV file content:")
            print("-" * 40)
            print(csv_content)
            print("-" * 40)
        except Exception as e:
            print(f"Could not display CSV content: {e}")

    else:
        print("⚠️ No companies were extracted from the text.")

except Exception as e:
    print(f"❌ Extraction failed: {e}")
    print("Please check your API key and internet connection.")

In [ ]:
# Download the generated CSV file
from google.colab import files

# Check if CSV file exists and download it
import os
if os.path.exists(CONFIG['output_csv']):
    print(f"📥 Downloading {CONFIG['output_csv']}...")
    files.download(CONFIG['output_csv'])
    print("✅ Download completed!")
else:
    print(f"❌ File {CONFIG['output_csv']} not found. Please run the extraction first.")